In [ ]:
import requests
import json
import pickle

## [abo | gun | blm]
campaign = 'blm'

url_active_17 = 'http://130.56.248.5:9357/active2017/_search?pretty&scroll=1m'
url_active_17_scroll = 'http://130.56.248.5:9357/_search/scroll?pretty'

with open('queries/{}_video.json'.format(campaign), 'r') as f:
    query_temp = f.read()
    
query_json = json.loads(query_temp)
query_str = json.dumps(query_json)

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

videos = []
scroll_ids = []
total = 0

r = requests.post(url_active_17, data=query_str, headers=headers)
result = json.loads(r.content.decode('utf-8'))
hits = result['hits']['hits']
print("# returned hits:", len(hits))
num_total_hits = result['hits']['total']
print("# total hits:", num_total_hits)
print("max_score:", result['hits']['max_score'])
print("=============================================")
last_scroll_id = result['_scroll_id']
scroll_ids.append(last_scroll_id)
total += len(hits)

for hit in hits:
    videos.append(hit)

## Scroll search
while len(hits) > 0:
    try:
        scroll_query_json = {
            "scroll" : "1m", 
            "scroll_id" : last_scroll_id
        }
        
        scroll_query_str = json.dumps(scroll_query_json)
        r = requests.post(url_active_17_scroll, data=scroll_query_str, headers=headers)
        result = json.loads(r.content.decode('utf-8'))
        hits = result['hits']['hits']
        print("returned hits:", len(hits))
        total += len(hits)
        print("# total:", total)
        last_scroll_id = result['_scroll_id']
        scroll_ids.append(last_scroll_id)
        for hit in hits:
            videos.append(hit)
    except:
        #print(result)
        print('Error...')
        break


print("# videos:", len(videos))
print("# total:", total)

results = {}
for video in videos:
    videoId = video['_source']['id']
    results[videoId] = video

print("# results: ", len(results.keys()))

try:
    pickle.dump(results, open("../data/social_media/{}/videos_by_keywords.pkl".format(campaign), "wb"))
except:
    print("couldn't dump the videos!!!")
    pass

## Clear/Close the sessions
clear_scroll_query_json = {"scroll_id": scroll_ids}
clear_scroll_query_str = json.dumps(clear_scroll_query_json)
r = requests.delete(url_active_17_scroll, data=clear_scroll_query_str, headers=headers)
